In [ ]:
#İsmail Burak Kurhan
#220201055
import numpy as np
import cv2
from random import randint
import math
from numpy.linalg import inv

def findInlier(homography,l_kp1,l_kp2):
    returnSrc=[]
    returnDst=[]
    distance=3.0
    temp=0
    control =[]
    for x in range(len(l_kp1)):
      
        pointsList=np.dot(homography,np.array([[l_kp1[x][0]],[l_kp1[x][1]],[1]]))
        pointsList[0]=pointsList[0]/pointsList[2]
        pointsList[1]=pointsList[1]/pointsList[2]
        pointsList=np.array(pointsList)
     
        flag = False
        for y in range(len(l_kp2)):
            d=float(math.sqrt((pointsList[0]-l_kp2[y][0])**2+(pointsList[1]-l_kp2[y][1])**2))
            if (d<3.0):
                if (distance>d):
                    distance=d
                    temp=y
                    flag = True
        if flag == True:
            if ( temp not in control):    
                returnSrc.append((l_kp1[x]))
                returnDst.append((l_kp2[temp]))
                control.append(temp)      
    return np.array(returnSrc),np.array(returnDst)


def dlt(src_points,dst_points):
    Average_x_1 =0
    Average_y_1 =0
    Average_x_2 =0
    Average_y_2 =0

    for i in range(len(src_points)):
        Average_x_1+=src_points[i][0]
        Average_y_1+=src_points[i][1]
        Average_x_2+=dst_points[i][0]
        Average_y_2+=dst_points[i][1]
 
    Average_x_1= Average_x_1 / len(src_points)
    Average_y_1= Average_y_1 / len(src_points)
    Average_x_2= Average_x_2 / len(src_points)
    Average_y_2= Average_y_2 / len(src_points)

    new_src_points=[]
    new_dst_points=[]

    for i in range(len(src_points)):

        x1=src_points[i][0]-Average_x_1
        y1=src_points[i][1]-Average_y_1
        x2=dst_points[i][0]-Average_x_2
        y2=dst_points[i][1]-Average_y_2
     
        new_src_points.append([x1,y1])
        new_dst_points.append([x2,y2])

    distancesrc=0
    distancedst=0

    for i in range(len(new_src_points)):
        distancesrc += float(math.sqrt(new_src_points[i][0]**2 +   new_src_points[i][1]**2))
        distancedst += float(math.sqrt(new_dst_points[i][0]**2 +   new_dst_points[i][1]**2))

    lamda=  math.sqrt(2)/float(distancesrc)
    lamda2=  math.sqrt(2)/float(distancedst)
    t=np.array([[float(lamda),0,float(lamda)*(-Average_x_1)],[0,float(lamda),float(lamda)*(-Average_y_1)],[0,0,1]])
    t2=np.array([[float(lamda2),0,float(lamda2)*(-Average_x_2)],[0,float(lamda2),float(lamda2)*(-Average_y_2)],[0,0,1]])
    list=[]
    for i in range(len(new_src_points)):
        new_src_points[i][0]= lamda * new_src_points[i][0]
        new_src_points[i][1]= lamda * new_src_points[i][1]
        new_dst_points[i][0]= lamda2 * new_dst_points[i][0]
        new_dst_points[i][1]= lamda2 * new_dst_points[i][1]
    for i in range(len(new_src_points)):
     
        list.append([0,0,0,-new_src_points[i][0],-new_src_points[i][1],-1,new_dst_points[i][1]*new_src_points[i][0],new_dst_points[i][1]*new_src_points[i][1],new_dst_points[i][1]])
        list.append([new_src_points[i][0],new_src_points[i][1],1,0,0,0,-new_dst_points[i][0]*new_src_points[i][0],-new_src_points[i][1]*new_dst_points[i][0],-new_dst_points[i][0]])
    list=np.array(list)
    w,u,vt=cv2.SVDecomp(np.asarray(list))
    homography=[[vt[-1][0],vt[-1][1],vt[-1][2]],[vt[-1][3],vt[-1][4],vt[-1][5]],[vt[-1][6],vt[-1][7],vt[-1][8]]]
    invT = inv(t2)

    newHomograpy=np.dot(invT,homography)

    return np.dot(newHomograpy,t)
#---------Main Starts Here-----------------

img1 = cv2.imread('img1.jpg')          
img2 = cv2.imread('img2.jpg')

sift = cv2.xfeatures2d.SIFT_create(contrastThreshold = 0.05 ,edgeThreshold = 10)

kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)
best = 999999
temp_for_l=0
keypoints_1 = []
keypoints_2 = []
difference = 0
distance_point= 0

for i in range (len(des1)):
    for l in range (len(des2)):
        for m in range (len(des1[0])):
            difference += (des1[i][m]-des2[l][m])**2
        distance_point = math.sqrt(difference)
        difference =0
        if (distance_point<best):
            best=distance_point
            temp_for_l = l
    print ("Matches are calculating, please wait... %",i/len(des1)*100 )
    if (kp1[i].pt not in keypoints_1 and kp2[temp_for_l].pt not in keypoints_2 ):
        keypoints_1.append((kp1[i].pt))
        keypoints_2.append((kp2[temp_for_l].pt))
    temp_for_l = 0
    best = 999999
    
Nransac=10000
n=0
numberOfinliersCopy=0
lastSrc=[]
lastDst=[]
Inlier=[]

keypoints_1=np.array(keypoints_1)
keypoints_2=np.array(keypoints_2)
#------------------RANSAC Starts here----------------------
best_inlier=0
while(n<Nransac):
 
    numberOfinliers=0
    random=np.random.randint(0,len(keypoints_1))
    random2=np.random.randint(0,len(keypoints_1))
    random3=np.random.randint(0,len(keypoints_1))
    random4=np.random.randint(0,len(keypoints_1))
 
 
    src_points=[(keypoints_1[random]),(keypoints_1[random2]),(keypoints_1[random3]),(keypoints_1[random4])]
    dst_points=[(keypoints_2[random]),(keypoints_2[random2]),(keypoints_2[random3]),(keypoints_2[random4])]
 
    src_points=np.array(src_points)
    dst_points=np.array(dst_points)

    Homography2=dlt(src_points,dst_points)
 
    for x in range(len(keypoints_1)):
        pointsList=np.dot(Homography2,np.array([[keypoints_1[x][0]],[keypoints_1[x][1]],[1]]))
   
        pointsList[0]=pointsList[0]/pointsList[2]
        pointsList[1]=pointsList[1]/pointsList[2]
        pointsList[2]=pointsList[2]/pointsList[2]
        pointsList=np.array(pointsList)
     

        for y in range (len(keypoints_2)):
           
            if (math.sqrt((pointsList[0]-keypoints_2[y][0])**2+(pointsList[1]-keypoints_2[y][1])**2)<=3.0):
                numberOfinliers+=1
                break
                 
    inlierRatio= float(numberOfinliers / len(keypoints_1))
    if(numberOfinliers>numberOfinliersCopy):
            homography3=Homography2
            best_inlier=numberOfinliers
            Nransac=int((-2.0/float(math.log10(1.0-(float(inlierRatio**4))))) - 1.0)
            n=0
 
    n+=1
    numberOfinliersCopy=best_inlier
    print ("RANSAC number is : ",Nransac)
    print ("Best Number of inlier is",best_inlier)

print ("best inlier Ratio is:",(best_inlier/len(keypoints_1))*100)

Src=[]
Dst=[]
Src,Dst=findInlier(homography3,keypoints_1,keypoints_2)  # Find inlier with best homography

n=0

#------------------ loop with dlt to find better inliers-------------------
previousSrc=[]
previousDst=[]
while(1):
 
    betterHomo=dlt(Src,Dst)

    previousSrc=np.copy(Src)
    previousDst=np.copy(Dst)

    Src = []
    Dst = []
    if (len(previousSrc) <= 4 or  len(previousDst) <= 4):
        break
    Src,Dst=findInlier(betterHomo,previousSrc,previousDst)
   
    if (len(Src)==len(previousSrc)): # When Src = scrcopy inliers are converge
        break
        
        
#--------------Draw rectangle-----------------

bottom_right=np.dot(betterHomo,np.array([[455.0],[576.0],[1.0]]))
top_left=np.dot(betterHomo,np.array([[160.0],[160.0],[1.0]]))

top_left[0]=float(top_left[0]/top_left[2])
top_left[1]=float(top_left[1]/top_left[2])

bottom_right[0]=float(bottom_right[0]/bottom_right[2])
bottom_right[1]=float(bottom_right[1]/bottom_right[2])


detection = cv2.imread('img2.jpg')

cv2.rectangle(detection,(top_left[0],top_left[1]),(bottom_right[0],bottom_right[1]),(0,0,255),5)

cv2.imwrite("object detection.png",detection)

#Comment-1: SIFT algortihm: SIFT firstly use difference of gaussian to estimate minimum and maximum scale space.
#Then determine keypoints candidates by eliminating low contrast points. After do keypoint orientation with first image gradients
# finaly compute  descriptors with gradient  magnitude  and  orientation
# ORB algorithm: ORB is using FAST keypoint detector and BRIEF descriptor.First determine the key points with using FAST.
# FAST does not compute the orientation and rotation variant. It works with intensity differences between corners and center.
# Then BRIEF descriptor calculate descriptors.
# SIFT is  more scale invariant than ORB algorithm.
# According to some researches on internet ORB algorithm much faster than SIFT algorithm but often SIFT algorithm find more keypoints
# then ORB. ORB is faster than SIFT because ORB use hamming distance but SIFT use euclidian distance and SIFT algorithm do much
# calculation and using more memory with 128-d descriptors.

# Comment-2: I am using SIFT algorithm and i found nearest neighbour for each correspondence with euclidean distance.
#SIFT descriptors are 128 dimensions. To find nearest neighbour in 128-d descriptor we must use euclidean distance
# If a used ORB algortihm. ORB descripters are binery and to find nearest neighbour in ORB descriptor we must use hamming distance
# Hamming distance gives difference between two binary numbers with xor


# Comment-3: Our aim is in this part find potential inliers and find a inlier ratio to determine which homogrophy is better
# If we set Euclidean distance threshold to smaller than the 3 pixels. we get less inliers and inlier ratio so our inlier ratio
# will decrease. Ransac run time increase but maybe we can lost some good correspondences.

# Comment-4 : Normalization is putting both points on the border which makes the solution more stable and puts it near from the center.
# The source and target coordinate data are usually noisy without normalization. With normalization we solve this problem.
# Basically normalization provides better handled points and calculations.